In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from tensorflow import keras
from tensorflow.keras import layers, models
from skimage.feature import hog

In [2]:
data_path = "/content/drive/MyDrive/dataset"

In [3]:
def load_images(data_path, categories, img_size=(64, 64)):
    images = []
    labels = []

    for category in categories:
        category_path = os.path.join(data_path, category)
        label = categories.index(category)  # Assign labels as indices (0, 1, 2, ...)

        for img_file in os.listdir(category_path):
            img_path = os.path.join(category_path, img_file)
            try:
                # Load image, resize, and normalize
                img = cv2.imread(img_path)
                img = cv2.resize(img, img_size)
                img = img / 255.0  # Normalize to range [0, 1]
                images.append(img)
                labels.append(label)
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")

    return np.array(images), np.array(labels)

# Define categories based on folder names
categories = ['cloudy', 'foggy', 'rainy', 'shine', 'sunrise']


In [4]:
# Load images and labels
images, labels = load_images(data_path, categories)

# Split data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

print(f"Training set size: {len(X_train)}")
print(f"Testing set size: {len(X_test)}")


Error loading image /content/drive/MyDrive/dataset/rainy/rain141.jpg: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Error loading image /content/drive/MyDrive/dataset/shine/shine131.jpg: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Training set size: 1198
Testing set size: 300


In [5]:
# Define CNN model
def create_cnn(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Create and compile the CNN
cnn = create_cnn(X_train[0].shape, len(categories))
cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
history = cnn.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 9s 98ms/step - accuracy: 0.4183 - loss: 1.4030 - val_accuracy: 0.6633 - val_loss: 0.8606
Epoch 2/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6885 - loss: 0.8237 - val_accuracy: 0.7167 - val_loss: 0.7961
Epoch 3/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7201 - loss: 0.7554 - val_accuracy: 0.8000 - val_loss: 0.6036
Epoch 4/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7811 - loss: 0.5634 - val_accuracy: 0.8167 - val_loss: 0.5351
Epoch 5/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7984 - loss: 0.5392 - val_accuracy: 0.7833 - val_loss: 0.5559
Epoch 6/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8151 - loss: 0.4705 - val_accuracy: 0.7467 - val_loss: 0.7444
Epoch 7/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8123 - loss: 0.5108 - val_accuracy: 0.8467 - val_loss: 0.4699
Epoch 8/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8970 - loss: 0.3059 - val_accuracy: 0.8367 - val_los

In [8]:
from google.colab import files

uploaded = files.upload()

for file_name in uploaded.keys():

    img = cv2.imread(file_name)
    img = cv2.resize(img, (64, 64))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)

    prediction = cnn.predict(img)
    predicted_class = np.argmax(prediction, axis=1)[0]

    print(f"Predicted Category: {categories[predicted_class]}")


Saving images.jfif to images.jfif
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Predicted Category: rainy
